In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from scipy.stats import entropy
import numpy as np

from load_moonboard import load_moonboard


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--workers', type=int, help='number of data loading workers', default=0)
parser.add_argument('--batchSize', type=int, default=512, help='input batch size')
parser.add_argument('--imageSize', type=int, default=512, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=200, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=128)
parser.add_argument('--ndf', type=int, default=32)
parser.add_argument('--niter', type=int, default=70, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0001, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

opt = parser.parse_args([])

In [3]:
if opt.manualSeed is None:
    opt.manualSeed = random.randint(1, 10000)
print("Random Seed: ", opt.manualSeed)
random.seed(opt.manualSeed)
torch.manual_seed(opt.manualSeed)

cudnn.benchmark = True


"""Load the dataset"""
class MoonBoardDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train = True):
        self.train = train

        (self.x_train, self.y_train), (self.x_test, self.y_test) = load_moonboard()
        self.x_train = self.x_train.transpose(0,1,3,2 ).astype(float)
        self.x_test = self.x_test.transpose(0,1,3,2 ).astype(float)
        self.y_train = self.y_train.reshape(-1,1).astype(int)
        self.y_test = self.y_test.reshape(-1,1).astype(int)

    def __len__(self):
        if self.train:
            return len(self.x_train)
        else:
            return len(self.x_test)

    def __getitem__(self, idx):
        if self.train:
            return self.x_train[idx], self.y_train[idx]
        else:
            return self.x_test[idx], self.y_test[idx]

dataset = MoonBoardDataset(train = True)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=opt.batchSize,
                                         shuffle=True, num_workers=int(opt.workers))

nc = 3

Random Seed:  1957


In [4]:
print(torch.cuda.is_available())
device = torch.device("cuda")
ngpu = int(opt.ngpu)
nz = int(opt.nz)
ngf = int(opt.ngf)
ndf = int(opt.ndf)

True


In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.convtrans1 = nn.ConvTranspose2d(nz, ngf * 4, (5,4), 2, 0, bias=False)
        self.convtrans2 = nn.ConvTranspose2d(ngf * 4, ngf * 2, (5,4), 2, 2, bias=False)
        self.convtrans3 = nn.ConvTranspose2d(    ngf * 2,      nc, (4,3), 2, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(ngf * 4)
        self.bn2 = nn.BatchNorm2d(ngf * 2)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        
   

    def forward(self, x):
        x = self.convtrans1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.convtrans2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.convtrans3(x)
        x = self.relu(x)
        
        x = self.tanh(x)
        
        return x


netG = Generator(ngpu).to(device)
netG.apply(weights_init)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        
        self.leakyRelu = nn.LeakyReLU(0.2, inplace=True)
        self.bn1 = nn.BatchNorm2d(ndf * 2)
        self.bn2 = nn.BatchNorm2d(ndf * 4)
        self.bn3 = nn.BatchNorm2d(ndf * 8)
        self.conv1 = nn.Conv2d(nc, ndf, 4, 2, 1, bias=False)
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False)
        self.conv3 = nn.Conv2d(ndf * 2, 1, (5,4), 2, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.leakyRelu(x)
        
        x = self.conv2(x)
        x = self.bn1(x)
        x = self.leakyRelu(x)
        
        x = self.conv3(x)
        
        return x.view(-1, 1).squeeze(1)


netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)



Discriminator(
  (leakyRelu): LeakyReLU(negative_slope=0.2, inplace)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (conv3): Conv2d(64, 1, kernel_size=(5, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (sigmoid): Sigmoid()
)

In [7]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=13):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
def inception_score(imgs, inception_model, splits = 1):

    N = len(imgs)    
    split_scores = []
    
    preds = nn.Softmax(dim = 1)(inception_model(imgs))
    
    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = torch.mean(part, 0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx.cpu().detach().numpy(), py.cpu().detach().numpy()))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)
    
inception_model = torch.load("inception_model.trch").to(device)

In [8]:
criterion = nn.MSELoss()

fixed_noise = torch.randn(opt.batchSize, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))

In [9]:
ff_noise = torch.randn(1000, nz, 1, 1, device=device)

print("Inception score ", inception_score(netG(ff_noise), inception_model))

for epoch in range(opt.niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].type(torch.FloatTensor).to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        for _ in range(2):
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            fake = netG(noise)
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2), end="\r")
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (opt.outf, epoch),
                    normalize=True)
    print("")
    print("Inception score ", inception_score(netG(ff_noise), inception_model))

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (opt.outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (opt.outf, epoch))


Inception score  (1.1749289, 0.0)
[0/70][8/9] Loss_D: 1.4462 Loss_G: 0.0061 D(x): 0.0632 D(G(z)): 0.7051 / 1.0392
Inception score  (1.1419706, 0.0)
[1/70][8/9] Loss_D: 0.9726 Loss_G: 0.0100 D(x): 0.2874 D(G(z)): 0.5959 / 0.9875
Inception score  (1.1401275, 0.0)
[2/70][8/9] Loss_D: 0.7246 Loss_G: 0.0093 D(x): 0.4497 D(G(z)): 0.5486 / 0.9696
Inception score  (1.1853774, 0.0)
[3/70][8/9] Loss_D: 0.7532 Loss_G: 0.0126 D(x): 0.4570 D(G(z)): 0.5998 / 0.9283
Inception score  (1.1885377, 0.0)
[4/70][8/9] Loss_D: 0.7423 Loss_G: 0.0418 D(x): 0.4089 D(G(z)): 0.5572 / 0.8175
Inception score  (1.2034277, 0.0)
[5/70][8/9] Loss_D: 0.7162 Loss_G: 0.1172 D(x): 0.3501 D(G(z)): 0.5010 / 0.6651
Inception score  (1.2087543, 0.0)
[6/70][8/9] Loss_D: 0.5764 Loss_G: 0.1955 D(x): 0.4129 D(G(z)): 0.4215 / 0.5654
Inception score  (1.2083981, 0.0)
[7/70][8/9] Loss_D: 0.4933 Loss_G: 0.1998 D(x): 0.4701 D(G(z)): 0.4048 / 0.5653
Inception score  (1.1869347, 0.0)
[8/70][8/9] Loss_D: 0.4777 Loss_G: 0.3402 D(x): 0.4415

KeyboardInterrupt: 

In [16]:
print("Inception score ", inception_score(next(iter(dataloader))[0].type(torch.FloatTensor).to(device), inception_model))
print(len(dataset))

Inception score  (2.6598134, 0.0)
6717
